In [174]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
import operator

import time
import datetime
from datetime import date
from datetime import timedelta
from tqdm.notebook import tqdm

In [175]:
prices = yf.download('IMOEX.ME', '2012-01-01', '2020-01-01')

[*********************100%***********************]  1 of 1 completed


In [191]:
prices = yf.download('IMOEX.ME', '2018-01-01', '2021-01-01')

prices = np.log(prices / prices.shift(1))
prices = prices.reset_index()
prices = prices.drop(0)

prices['Day'] = pd.DatetimeIndex(prices['Date']).day
prices['Month'] = pd.DatetimeIndex(prices['Date']).month
prices['Year'] = pd.DatetimeIndex(prices['Date']).year


[*********************100%***********************]  1 of 1 completed


In [192]:
totm_df = pd.DataFrame(columns = prices.columns)
min_year = prices.Year.min()
max_year = prices.Year.max()

first_month = prices.iloc[0, :].Month
last_month = prices.iloc[-1, :].Month

current_last_month = 12
to_drop = []

for year in tqdm(range(min_year, max_year+1)):
    if year == max_year:
        current_last_month = last_month
    for month in range(first_month, current_last_month+1):
        one = prices[(prices.Month == month) & (prices.Year == year)].iloc[[3]]
        last = prices[(prices.Month == month) & (prices.Year == year)].iloc[[-2]]
        totm_df = pd.concat((pd.concat((totm_df, one)), last))
        to_drop.extend(list(one.index))
        to_drop.extend(list(last.index))
    first_month = 1


In [193]:
rest_df = prices.drop(to_drop, axis = 0)

In [197]:

five_days_return = totm_df['Adj Close'].sum()/totm_df.shape[0]
print('Дневная доходность в течение смены месяца равна', five_days_return)

rest_return = rest_df['Adj Close'].sum()/rest_df.shape[0]
print('Дневная доходность в остальное время равна', rest_return)

change = (five_days_return - rest_return)/abs(rest_return)
dif = (np.exp(five_days_return) - np.exp(rest_return))*100

totm = (np.exp(totm_df['Adj Close'].sum()))*100
print(totm, '%')

rest = (np.exp(rest_df['Adj Close'].sum()))*100
print(rest, '%')

sum_dif = (np.exp(totm_df['Adj Close'].sum()) - np.exp(rest_df['Adj Close'].sum()))*100
print(sum_dif)


print('Больше в', change, 'раз')
print('Больше на', dif, '% дневная доходность')
rest_df.shape[0]

Дневная доходность в течение смены месяца равна 0.00015432529658716986
Дневная доходность в остальное время равна 0.0006186356252116177
101.11733824759199 %
151.07858743277407 %
-49.961249185182076
Больше в -0.7505392668998337 раз
Больше на -0.04644898143475462 % дневная доходность


667

In [91]:
# Тесты Флигнера-Килина о равенстве дисперсий
Fligner_1 = stats.fligner(totm_df['Close'],rest_df['Close'])
# print(Fligner_1[1],Fligner_2[1],Fligner_3[1])

if Fligner_1[1] >= 0.01:
    equal_1 = True
else:
    equal_1 = False

# Критерий t-Стьюдента
Ttest_12_all = stats.ttest_ind(totm_df['Close'],rest_df['Close'])
print(Ttest_12_all)
if Ttest_12_all[1] <= 0.1:
    print(('январь значимо больше'))
else:
    print('-')

Ttest_indResult(statistic=1.1882053478165755, pvalue=0.23492096249801817)
-


# Без теста Стьюдента

In [102]:
ticker = ('SBER','GAZP','ROSN','NVTK','GMKN','LKOH','PLZL','SIBN','SNGSP',
          'NLMK','TATN','CHMF','ALRS','MTSS','MAGN',
          'MGNT','IRAO','VTBR','PHOR','PIKK',
          'RSTI','HYDR','RTKM','FEES','AFKS','BANE','VSMO','AKRN',
          'UPRO','MGTS','AFLT','NMTP','NKNC','KZOS','MVID',
          'ROSB','RASP','APTK','LSRG','AVAN',
          'MSNG','OGKB','GCHE','CHEP','MSRS','TRMK','IRGZ',
          'IRKT','INGR','ALNU','KMAZ','LSNG','YAKG','TGKA',
          'SELG','KAZT','MRKS','FESH','MTLR','ENRU','MFGS',
          'MSTT','MRKP','MRKK','BSPB','USBN','UTAR','AQUA','KUBE')
year = []
day_increase = []
day_growth = []
growth = []
sum_totm = []
sum_rest = []

for i in range (len(ticker)):
    # скачивание данных
    prices = yf.download(ticker[i]+'.ME', '2012-01-01', '2018-01-01')
    
    prices = np.log(prices / prices.shift(1))
    prices = prices.reset_index().dropna()
    
    prices['Day'] = pd.DatetimeIndex(prices['Date']).day
    prices['Month'] = pd.DatetimeIndex(prices['Date']).month
    prices['Year'] = pd.DatetimeIndex(prices['Date']).year.dropna()
    
    totm_df = pd.DataFrame(columns = prices.columns)
    min_year = prices.Year.min()
    max_year = prices.Year.max()

    first_month = prices.iloc[0, :].Month
    last_month = prices.iloc[-1, :].Month

    current_last_month = 12
    to_drop = []

    for year in tqdm(range(min_year, max_year+1)):
        if year == max_year:
            current_last_month = last_month
        for month in range(first_month, current_last_month+1):
            one = prices[(prices.Month == month) & (prices.Year == year)].iloc[:5]
            last = prices[(prices.Month == month) & (prices.Year == year)].iloc[-3:]
            totm_df = pd.concat((pd.concat((totm_df, one)), last))
            to_drop.extend(list(one.index))
            to_drop.extend(list(last.index))
        first_month = 1
        
    rest_df = prices.drop(to_drop, axis = 0)

    five_days_return = totm_df['Close'].sum()/totm_df.shape[0]
    rest_return = rest_df['Close'].sum()/rest_df.shape[0]
    
    change = (five_days_return - rest_return)/abs(rest_return)
    day_increase.append(change)
    
    dif = (np.exp(five_days_return) - np.exp(rest_return))*100
    day_growth.append(dif)
    
    totm = (np.exp(totm_df['Close'].sum()))*100
    sum_totm.append(totm)

    rest = (np.exp(rest_df['Close'].sum()))*100
    sum_rest.append(rest)
    
    sum_dif = (np.exp(totm_df['Close'].sum()) - np.exp(rest_df['Close'].sum()))*100
    growth.append(sum_dif)


dict = {'ticker':ticker,
        'day_increase':day_increase,
       'day_growth':day_growth,
        'growth':growth,
        'sum_totm':sum_totm,
        'sum_rest':sum_rest}

comp = pd.DataFrame(dict)
comp

Exception in thread Thread-2296:
Traceback (most recent call last):
  File "C:\Users\nasty\anaconda3\lib\site-packages\urllib3\connection.py", line 159, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\nasty\anaconda3\lib\site-packages\urllib3\util\connection.py", line 61, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "C:\Users\nasty\anaconda3\lib\socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11001] getaddrinfo failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\nasty\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 670, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\nasty\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 381, in _make_request
    self._validate_conn(conn)
  File "C:\Users\nasty\a

KeyboardInterrupt: 

In [101]:
comp.mean()

day_increase      5.728605
day_growth        0.117947
growth           71.323062
sum_totm        186.907697
sum_rest        115.584635
dtype: float64

# Расчеты с тестом Стьюдента

In [93]:
ticker = ('SBER','GAZP','ROSN','NVTK','GMKN','LKOH','PLZL','SIBN','SNGSP',
          'NLMK','TATN','CHMF','ALRS','MTSS','MAGN',
          'MGNT','IRAO','VTBR','PHOR','PIKK',
          'RSTI','HYDR','RTKM','FEES','AFKS','BANE','VSMO','AKRN',
          'UPRO','MGTS','AFLT','NMTP','NKNC','KZOS','MVID',
          'ROSB','RASP','APTK','LSRG','AVAN',
          'MSNG','OGKB','GCHE','CHEP','MSRS','TRMK','IRGZ',
          'IRKT','INGR','ALNU','KMAZ','LSNG','YAKG','TGKA',
          'SELG','KAZT','MRKS','FESH','MTLR','ENRU','MFGS',
          'MSTT','MRKP','MRKK','BSPB','USBN','UTAR','AQUA','KUBE')
year = []
day_increase = []
day_growth = []
growth = []
norm = []
ttest = []

for i in range (len(ticker)):
    # скачивание данных
    prices = yf.download(ticker[i]+'.ME', '2012-01-01', '2018-01-01')
    
    prices = np.log(prices / prices.shift(1))
    prices = prices.reset_index().dropna()
    
    prices['Day'] = pd.DatetimeIndex(prices['Date']).day
    prices['Month'] = pd.DatetimeIndex(prices['Date']).month
    prices['Year'] = pd.DatetimeIndex(prices['Date']).year.dropna()
    
    totm_df = pd.DataFrame(columns = prices.columns)
    min_year = prices.Year.min()
    max_year = prices.Year.max()

    first_month = prices.iloc[0, :].Month
    last_month = prices.iloc[-1, :].Month

    current_last_month = 12
    to_drop = []

    for year in tqdm(range(min_year, max_year+1)):
        if year == max_year:
            current_last_month = last_month
        for month in range(first_month, current_last_month+1):
            one = prices[(prices.Month == month) & (prices.Year == year)].iloc[:3]
            last = prices[(prices.Month == month) & (prices.Year == year)].iloc[-2:]
            totm_df = pd.concat((pd.concat((totm_df, one)), last))
            to_drop.extend(list(one.index))
            to_drop.extend(list(last.index))
        first_month = 1
        
    rest_df = prices.drop(to_drop, axis = 0)

    five_days_return = totm_df['Close'].sum()/totm_df.shape[0]
    rest_return = rest_df['Close'].sum()/rest_df.shape[0]
    
    change = (five_days_return - rest_return)/abs(rest_return)
    day_increase.append(change)
    dif = (np.exp(five_days_return) - np.exp(rest_return))*100
    day_growth.append(dif)
    sum_dif = (np.exp(totm_df['Close'].sum()) - np.exp(rest_df['Close'].sum()))*100
    growth.append(sum_dif)
    
        # Проверка на нормальность
    Kolm = stats.kstest(totm_df['Close'],'norm',(stats.norm.fit(totm_df['Close'])))
    if Kolm[1]>=0.05:
        norm.append('+')
    else:
        norm.append('-')
        
    # Тесты Флигнера-Килина о равенстве дисперсий
    Fligner_1 = stats.fligner(totm_df['Close'],rest_df['Close'])    
    if Fligner_1[1] >= 0.05:
        equal_1 = True
    else:
        equal_1 = False
        
    # Критерий t-Стьюдента
    Ttest_12_all = stats.ttest_ind(totm_df['Close'],rest_df['Close'], equal_var = False)
    if Ttest_12_all[1] <= 0.05:
        ttest.append('доходности отличается')
    else:
        ttest.append('-')

dict = {'ticker':ticker,
        'day_increase':day_increase,
       'day_growth':day_growth,
        'growth':growth,
        'norm':norm,
       'ttest':ttest}

comp = pd.DataFrame(dict)
comp

[*********************100%***********************]  1 of 1 completed


<ipython-input-93-8ed5658dffbe>:22: RuntimeWarning: divide by zero encountered in log
  prices = np.log(prices / prices.shift(1))



[*********************100%***********************]  1 of 1 completed



[*********************100%***********************]  1 of 1 completed



[*********************100%***********************]  1 of 1 completed


...


[*********************100%***********************]  1 of 1 completed



[*********************100%***********************]  1 of 1 completed



[*********************100%***********************]  1 of 1 completed



[*********************100%***********************]  1 of 1 completed


,ticker,day_increase,day_growth,growth,norm,ttest
0,SBER,3.157489,0.125253,24.226574,-,-
1,GAZP,2.950136,0.186095,106.780015,-,-
2,ROSN,10.294377,0.144355,74.521383,+,-
3,NVTK,5.202014,0.083849,23.243494,-,-
4,GMKN,6.622526,0.134858,48.820690,-,-
...,...,...,...,...,...,...
64,BSPB,2.114372,0.080725,51.691278,-,-
65,USBN,2.325422,0.151423,86.922805,-,-
66,UTAR,-3.150167,-0.081707,-6.821329,-,-
67,AQUA,0.654555,0.019734,-19.181902,-,-


In [94]:
comp.mean()

day_increase    10.310296
day_growth       0.128313
growth          25.148225
dtype: float64